# Simple CNN on MNIST - Generalization

In [1]:
import numpy as np

import os

import keras
from keras import backend
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import load_model

import tensorflow as tf

from keras.callbacks import TensorBoard, EarlyStopping

from cleverhans.utils_mnist import data_mnist
from cleverhans.attacks import (BasicIterativeMethod, CarliniWagnerL2, DeepFool, ElasticNetMethod, 
                                FastFeatureAdversaries, FastGradientMethod, LBFGS, MadryEtAl, 
                                MomentumIterativeMethod, SPSA, SaliencyMapMethod, VirtualAdversarialMethod)
from cleverhans.utils_keras import KerasModelWrapper

from ipywidgets import interact
import ipywidgets as widgets

/home/Chris/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

In [3]:
from_saved_model = False
run_attack = False

## Configurable Parameters

In [4]:
# Used in Getting the Data
train_start=0
train_end=60000
test_start=0
test_end=10000

attack_start=0
attack_end=100

batch_size = 128
num_classes = 10
epochs = 500
input_shape = (28, 28, 1)

gen_steps = 11

num_points = 10
attack_names = ['basic_iterative', 'fast_gradient', 'madry', 'momentum_iterative']

run_ident = '18'

## Setup Tensorflow Session

In [5]:
keras.layers.core.K.set_learning_phase(0)

# Set TF random seed to improve reproducibility
tf.set_random_seed(1234)

# Create TF session and set as Keras backend session
sess = tf.Session()
keras.backend.set_session(sess)

## Get the MNIST Data

In [6]:
X_train, Y_train, X_test, Y_test = data_mnist(train_start=train_start,
                                              train_end=train_end,
                                              test_start=test_start,
                                              test_end=test_end)

Extracting /tmp/train-images-idx3-ubyte.gz
Extracting /tmp/train-labels-idx1-ubyte.gz
Extracting /tmp/t10k-images-idx3-ubyte.gz
Extracting /tmp/t10k-labels-idx1-ubyte.gz
X_train shape: (60000, 28, 28, 1)
X_test shape: (10000, 28, 28, 1)


## Create the CNN Architecture

In [7]:
# Define input TF placeholder
x = tf.placeholder(tf.float32, shape=(None, 28, 28, 1))
y = tf.placeholder(tf.float32, shape=(None, 10))

# Define TF model graph

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

preds = model(x)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

## Setup Adversarial Generation Class

In [8]:
class GenAdv(object):
    def __init__(self, session):
        self.session = session
        self.model = None
        self.data = None
        self.labels = None
    
    def evaluate_model(self, model, data, labels, attack_names, num_points=10):
        self.model = model
        self.data = data
        self.labels = labels
        
        attack_strengths = np.linspace(0, 0.5, num_points)
        
        losses = np.zeros((len(attack_names), num_points))
        accuracies = np.zeros((len(attack_names), num_points))
        
        for index_name, attack_name in enumerate(attack_names):
            print('Running attack: {}'.format(attack_name))
            for index_strength, attack_strength in enumerate(attack_strengths):
                print('Using attack strength: {}'.format(attack_strength))
                loss, accuracy = self.run_attack(attack_name, attack_strength)
                losses[index_name, index_strength] = loss
                accuracies[index_name, index_strength] = accuracy
            
        return losses, accuracies
    
    def run_attack(self, attack_name, attack_strength):
        wrap = KerasModelWrapper(self.model)
        
        if attack_name is 'basic_iterative':
            attack = BasicIterativeMethod(wrap, sess=self.session)
            attack_params = {'eps': attack_strength, # Default O.3
                             'eps_iter': 0.05,
                             'nb_iter': 10,
                             'y': self.labels,
                             'ord': np.inf,
                             'clip_min': None,
                             'clip_max': None}
        elif attack_name is 'carlini_wagner':
            attack = CarliniWagnerL2(wrap, sess=self.session)
            attack_params = {'y': self.labels,
                             'nb_classes': None,
                             'batch_size': 1,
                             'confidence': attack_strength, # Default 0
                             'learning_rate': 0.005,
                             'binary_search_steps': 5,
                             'max_iterations': 1000,
                             'abort_early': True,
                             'initial_const': 0.01,
                             'clip_min': 0,
                             'clip_max': 1}
        elif attack_name is 'deep_fool':
            attack = DeepFool(wrap, sess=self.session)
            attack_params = {'nb_candidate': attack_strength, # Default 10, INT
                             'overshoot': 0.02,
                             'max_iter': 50,
                             'nb_classes': None,
                             'clip_min': 0.0,
                             'clip_max': 1.0}
        elif attack_name is 'elastic_net':
            attack = ElasticNetMethod(wrap, sess=self.session)
            attack_params = {'y': self.labels,
                             'nb_classes': None,
                             'fista': True,
                             'beta': 0.001,
                             'decision_rule': 'EN',
                             'batch_size': 1,
                             'confidence': attack_strength, # Default 0
                             'learning_rate': 0.01,
                             'binary_search_steps': 9,
                             'max_iterations': 1000,
                             'abort_early': False,
                             'initial_const': 0.001,
                             'clip_min': 0,
                             'clip_max': 1}
        elif attack_name is 'fast_feature':
            attack = FastFeatureAdversaries(wrap, sess=self.session)
            attack_params = {'eps': attack_strength, # Default 0.3
                             'eps_iter': 0.05,
                             'nb_iter': 10,
                             'ord': np.inf,
                             'clip_min': None,
                             'clip_max': None}
        elif attack_name is 'fast_gradient':
            attack = FastGradientMethod(wrap, sess=self.session)
            attack_params = {'eps': attack_strength, # Default 0.3
                             'ord': np.inf,
                             'y': self.labels,
                             'clip_min': None,
                             'clip_max': None}
        elif attack_name is 'lbfgs':
            attack = LBFGS(wrap, sess=self.session)
            attack_params = {'batch_size': 1,
                             'binary_search_steps': 5,
                             'max_iterations': 1000,
                             'initial_const': attack_strength, # Default 0.01
                             'clip_min': 0,
                             'clip_max': 1}
        elif attack_name is 'madry':
            attack = MadryEtAl(wrap, sess=self.session)
            attack_params = {'eps': attack_strength, # Default 0.3
                             'eps_iter': 0.01,
                             'nb_iter': 40,
                             'y': self.labels,
                             'ord': np.inf,
                             'clip_min': None,
                             'clip_max': None,
                             'rand_init': True}
        elif attack_name is 'momentum_iterative':
            attack = MomentumIterativeMethod(wrap, sess=self.session)
            attack_params = {'eps': attack_strength, # Default 0.3
                             'eps_iter': 0.06,
                             'nb_iter': 10,
                             'y': self.labels,
                             'ord': np.inf,
                             'decay_factor': 1.0,
                             'clip_min': None,
                             'clip_max': None}
        elif attack_name is 'spsa':
            attack = SPSA(wrap, sess=self.session)
            attack_params = {'y': self.labels,
                             'epsilon': attack_strength, # Default None
                             'num_steps': None,
                             'is_targeted': False,
                             'early_stop_loss_threshold': None,
                             'learning_rate': 0.01,
                             'delta': 0.01,
                             'batch_size': 128,
                             'spsa_iters': 1,
                             'is_debug': False}
        elif attack_name is 'saliency_map':
            attack = SaliencyMapMethod(wrap, sess=self.session)
            attack_params = {'theta': attack_strength, # Default 1.0
                             'gamma': 1.0,
                             'nb_classes': None,
                             'clip_min': 0.0,
                             'clip_max': 1.0,
                             'symbolic_impl': True}
        elif attack_name is 'virtual_adversarial':
            attack = VirtualAdversarialMethod(wrap, sess=self.session)
            attack_params = {'eps': attack_strength, # Default 2.0
                             'num_iterations': 1,
                             'xi': 1e-06,
                             'clip_min': None,
                             'clip_max': None}
        else:
            raise ValueError('Invalid Attack Name!')
        
        adv_x = attack.generate(x, **attack_params)
        data_adv = adv_x.eval(feed_dict={x:self.data}, session=self.session)
        
        score = model.evaluate(data_adv, self.labels, verbose=0)
        loss = score[0]
        accuracy = score[1]
        
        return loss, accuracy

## Impact of Epoch on Security

In [9]:
def reset_weights(model):
    for layer in model.layers: 
        if hasattr(layer, 'kernel_initializer'):
            layer.kernel.initializer.run(session=sess)

In [10]:
# Initialize the Fast Gradient Sign Method (FGSM) attack object and graph

gen_adv = GenAdv(sess)

losses = np.zeros((gen_steps, len(attack_names), num_points))
accuracies = np.zeros((gen_steps, len(attack_names), num_points))

directory = 'model_r{}'.format(run_ident)
if not from_saved_model and not os.path.exists(directory):
    os.mkdir(directory)

for gen in range(10, gen_steps):
    print('Gen: {}'.format(gen))
    
    #callbacks = [TensorBoard(log_dir='./logs/overfitting_r{}_g{}'.format(run_ident, gen)),
    #           EarlyStopping(monitor='loss', min_delta=1e-5, patience=5, verbose=0, mode='auto')] # 0 - 6
    #callbacks = [TensorBoard(log_dir='./logs/overfitting_r{}_g{}'.format(run_ident, gen)),
    #           EarlyStopping(monitor='loss', min_delta=1e-5, patience=30, verbose=0, mode='auto')] # 6 - 9
    callbacks = [TensorBoard(log_dir='./logs/overfitting_r{}_g{}'.format(run_ident, gen)),
                EarlyStopping(monitor='loss', min_delta=1e-5, patience=60, verbose=0, mode='auto')] # 10
    
    # Shuffle Labels
    prob_sel = gen / (gen_steps-1)
    print('Proability of Randomization: {}'.format(prob_sel))
    
    # Extract Sample to Edit
    Y_rand = np.copy(Y_train)
    sel = np.random.choice([False, True], size=(Y_rand.shape[0],), p=(1-prob_sel, prob_sel))
    Y_sel = Y_rand[sel,:]
    
    np.random.shuffle(Y_sel)
    
    # Debug
    rand_equal = np.equal(Y_rand[sel,:], Y_sel)
    rand_mean = np.mean(np.all(rand_equal, axis=1))
    print('Rand mean (only shuffled): {}'.format(rand_mean))
    # Debug
    
    Y_rand[sel,:] = np.copy(Y_sel)
    
    # Debug
    rand_equal = np.equal(Y_rand[sel,:], Y_sel)
    rand_mean = np.mean(np.all(rand_equal, axis=1))
    print('Rand mean (Should be 1): {}'.format(rand_mean))
    # Debug
    
    # Debug
    rand_equal = np.equal(Y_rand, Y_train)
    rand_mean = np.mean(np.all(rand_equal, axis=1))
    print('Rand mean (all): {}'.format(rand_mean))
    # Debug
    
    if from_saved_model:
        model = load_model('model_r{}/model_g{}.h5'.format(run_ident, gen))
    else:
        reset_weights(model)
        model.fit(X_train, Y_rand,
                  batch_size=batch_size,
                  epochs=epochs,
                  verbose=1,
                  validation_data=(X_test, Y_test),
                  callbacks=callbacks)
        model.save('model_r{}/model_g{}.h5'.format(run_ident, gen))

    if run_attack:
        loss, accuracy = gen_adv.evaluate_model(model, X_test[attack_start:attack_end, :], 
                                                Y_test[attack_start:attack_end], 
                                                attack_names, num_points=num_points)
        losses[gen,:,:] = loss
        accuracies[gen,:,:] = accuracy

Gen: 10
Proability of Randomization: 1.0
Rand mean (only shuffled): 0.10108333333333333
Rand mean (Should be 1): 1.0
Rand mean (all): 0.10108333333333333
Train on 60000 samples, validate on 10000 samples
Epoch 1/500
60000/60000 [==============================] - 4s 75us/step - loss: 2.3020 - acc: 0.1116 - val_loss: 2.3003 - val_acc: 0.1135
Epoch 2/500
60000/60000 [==============================] - 3s 57us/step - loss: 2.3012 - acc: 0.1123 - val_loss: 2.2942 - val_acc: 0.1135
Epoch 3/500
60000/60000 [==============================] - 3s 57us/step - loss: 2.3004 - acc: 0.1128 - val_loss: 2.2948 - val_acc: 0.1148
Epoch 4/500
60000/60000 [==============================] - 3s 57us/step - loss: 2.2993 - acc: 0.1154 - val_loss: 2.2841 - val_acc: 0.1428
Epoch 5/500
60000/60000 [==============================] - 3s 57us/step - loss: 2.2962 - acc: 0.1190 - val_loss: 2.2862 - val_acc: 0.1409
Epoch 6/500
60000/60000 [==============================] - 3s 57us/step - loss: 2.2911 - acc: 0.1261 - val

60000/60000 [==============================] - 3s 57us/step - loss: 5.7267e-05 - acc: 1.0000 - val_loss: 11.9276 - val_acc: 0.1015
Epoch 59/500
60000/60000 [==============================] - 3s 57us/step - loss: 5.3644e-05 - acc: 1.0000 - val_loss: 11.9544 - val_acc: 0.1012
Epoch 60/500
60000/60000 [==============================] - 3s 57us/step - loss: 5.0875e-05 - acc: 1.0000 - val_loss: 11.9563 - val_acc: 0.1011
Epoch 61/500
60000/60000 [==============================] - 3s 57us/step - loss: 4.8118e-05 - acc: 1.0000 - val_loss: 11.9709 - val_acc: 0.1014
Epoch 62/500
60000/60000 [==============================] - 3s 57us/step - loss: 4.5896e-05 - acc: 1.0000 - val_loss: 11.9801 - val_acc: 0.1013
Epoch 63/500
60000/60000 [==============================] - 3s 57us/step - loss: 4.3787e-05 - acc: 1.0000 - val_loss: 11.9933 - val_acc: 0.1014
Epoch 64/500
60000/60000 [==============================] - 3s 57us/step - loss: 4.1809e-05 - acc: 1.0000 - val_loss: 11.9995 - val_acc: 0.1013
Epoch

Epoch 115/500
60000/60000 [==============================] - 3s 57us/step - loss: 1.3156e-05 - acc: 1.0000 - val_loss: 12.2573 - val_acc: 0.1017
Epoch 116/500
60000/60000 [==============================] - 3s 57us/step - loss: 1.2983e-05 - acc: 1.0000 - val_loss: 12.2599 - val_acc: 0.1013
Epoch 117/500
60000/60000 [==============================] - 3s 57us/step - loss: 1.2809e-05 - acc: 1.0000 - val_loss: 12.2640 - val_acc: 0.1015
Epoch 118/500
60000/60000 [==============================] - 3s 57us/step - loss: 1.2639e-05 - acc: 1.0000 - val_loss: 12.2688 - val_acc: 0.1019
Epoch 119/500
60000/60000 [==============================] - 3s 57us/step - loss: 1.2474e-05 - acc: 1.0000 - val_loss: 12.2722 - val_acc: 0.1019
Epoch 120/500
60000/60000 [==============================] - 3s 57us/step - loss: 1.2323e-05 - acc: 1.0000 - val_loss: 12.2737 - val_acc: 0.1018
Epoch 121/500
60000/60000 [==============================] - 3s 57us/step - loss: 1.2167e-05 - acc: 1.0000 - val_loss: 12.2761 - v

60000/60000 [==============================] - 3s 58us/step - loss: 7.4466e-06 - acc: 1.0000 - val_loss: 12.3736 - val_acc: 0.1017
Epoch 172/500
60000/60000 [==============================] - 3s 58us/step - loss: 7.3873e-06 - acc: 1.0000 - val_loss: 12.3756 - val_acc: 0.1016
Epoch 173/500
60000/60000 [==============================] - 4s 58us/step - loss: 7.3320e-06 - acc: 1.0000 - val_loss: 12.3768 - val_acc: 0.1016
Epoch 174/500
60000/60000 [==============================] - 3s 58us/step - loss: 7.2770e-06 - acc: 1.0000 - val_loss: 12.3771 - val_acc: 0.1016
Epoch 175/500
60000/60000 [==============================] - 3s 58us/step - loss: 7.2199e-06 - acc: 1.0000 - val_loss: 12.3796 - val_acc: 0.1016
Epoch 176/500
60000/60000 [==============================] - 3s 58us/step - loss: 7.1679e-06 - acc: 1.0000 - val_loss: 12.3814 - val_acc: 0.1016
Epoch 177/500
60000/60000 [==============================] - 3s 58us/step - loss: 7.1129e-06 - acc: 1.0000 - val_loss: 12.3828 - val_acc: 0.1016

In [11]:
if run_attack:
    np.save('loss_r{}.npy'.format(run_ident), losses)
    np.save('accuracy_r{}.npy'.format(run_ident), accuracies)
    max_loss = losses.flatten().max()
    max_accuracy = accuracies.flatten().max()

In [12]:
def plot_func(gen):
    fig = plt.figure(figsize=(8, 6))
    plt.subplot(121)
    for index, attack_name in enumerate(attack_names):
        x_plt = np.linspace(0, 0.5, num_points)
        y_plt = losses[epoch, index, :].flatten()
        plt.plot(x_plt, y_plt, label=attack_name)
    plt.title('Adversarial Loss')
    plt.xlabel('Attack Strength')
    plt.ylabel('Loss')
    plt.ylim(ymax=max_loss)
    plt.legend()

    plt.subplot(122)
    for index, attack_name in enumerate(attack_names):
        x_plt = np.linspace(0, 0.5, num_points)
        y_plt = accuracies[epoch, index, :].flatten()
        plt.plot(x_plt, y_plt, label=attack_name)
    plt.title('Adversarial Accuracy')
    plt.xlabel('Attack Strength')
    plt.ylabel('Accuracy')
    plt.ylim(ymax=max_accuracy)
    plt.legend()
    plt.show()

In [13]:
if run_attack:
    interact(plot_func, gen=widgets.IntSlider(min=0,max=gen_steps-1,step=1,value=0));